In [1]:
import requests
import pandas as pd
d = {1: 'EURUSD',
 1686: 'BTCUSDT',
 191: 'AAPL.NASDAQ',
 250: 'TSLA.NASDAQ',
 238: 'NVDA.NASDAQ'}

unity_dict = {
    "EURUSD": 47,
    "BTCUSD": 61343,
    "AAPL.NASDAQ": 8115,
    "TSLA.NASDAQ": 1291,
    "NVDA.NASDAQ": 4711

}


# EURUSD (47)

# BTCUSD (61343)

# AAPL.NASDAQ (8115)


# TSLA.NASDAQ (1291)

# NVDA.NASDAQ (4711)

reaL_value = dict(zip(d.keys(), unity_dict.values()))

reaL_value

{1: 47, 1686: 61343, 191: 8115, 250: 1291, 238: 4711}

# Сопоставляем id

In [9]:
import pandas as pd

# Чтение листов "US EQ" и "FX" из Excel-файла
file_path = 'ID инструментов (2).xlsx'
sheets = ["US EQ", "FX"]
dfs = pd.read_excel(file_path, sheet_name=sheets)

# Объединение DataFrame-ов по строкам
combined_df = pd.concat([dfs["US EQ"], dfs["FX"]], axis=0, ignore_index=True)

# Вывод объединённого DataFrame
(combined_df.head())


,№,ID,Ticket,Code,Name,Unnamed: 3
0,1,8115.0,AAPL,AAPL.NASDAQ,NaN,NaN
1,2,4711.0,NVDA,NVDA.NASDAQ,NaN,NaN
2,3,1370.0,MSFT,MSFT.NASDAQ,NaN,NaN
3,4,8191.0,AMZN,AMZN.NASDAQ,NaN,NaN
4,5,795.0,GOOG,GOOG.NASDAQ,NaN,NaN


In [20]:
# Чтение листов "US EQ" и "FX" из файла Excel
file_path = 'ID инструментов (2).xlsx'
sheets = ["US EQ", "FX"]
dfs = pd.read_excel(file_path, sheet_name=sheets)

# Извлечение DataFrame для каждого листа
df_us_eq = dfs["US EQ"]
df_fx = dfs["FX"]


print("\nЛист FX:")

(df_fx.head())


Лист FX:


,№,ID,Ticket,Unnamed: 3
0,1,47,EURUSD,SPOT
1,2,37,GBPUSD,SPOT
2,3,41,USDJPY,SPOT
3,4,30,AUDUSD,SPOT
4,5,64,USDCAD,TOM


In [21]:
unity_instruments = df_fx.copy()
unity_instruments.drop(['№','Unnamed: 3'], axis=1, inplace=True)
# unity_instruments = unity_instruments.dropna()
# # unity_instruments = unity_instruments.dropna()
# ticket = unity_instruments['Ticket'].dropna().str.strip().tolist()
# code = unity_instruments['Code'].tolist()
unity_instruments.head()

,ID,Ticket
0,47,EURUSD
1,37,GBPUSD
2,41,USDJPY
3,30,AUDUSD
4,64,USDCAD


In [11]:
unity_instruments = combined_df.copy()
unity_instruments.drop(['Name','Unnamed: 3'], axis=1, inplace=True)
unity_instruments = unity_instruments.dropna()
# unity_instruments = unity_instruments.dropna()
ticket = unity_instruments['Ticket'].dropna().str.strip().tolist()
code = unity_instruments['Code'].tolist()
unity_instruments.head()

,№,ID,Ticket,Code
0,1,8115.0,AAPL,AAPL.NASDAQ
1,2,4711.0,NVDA,NVDA.NASDAQ
2,3,1370.0,MSFT,MSFT.NASDAQ
3,4,8191.0,AMZN,AMZN.NASDAQ
4,5,795.0,GOOG,GOOG.NASDAQ


In [23]:
bondash_data = requests.get('https://clientapi.bondah.com/instrument').json()


In [49]:
bondash_data

[{'groupId': 16,
  'groupName': '₿ HighCap',
  'instruments': [{'groupId': 16,
    'siteName': 'Bitcoin',
    'turnOn': True,
    'description': 'BTCUSDT',
    'id': 1686,
    'name': 'BTCUSDT.fut'},
   {'groupId': 16,
    'siteName': 'Bit.Cash',
    'turnOn': True,
    'description': 'BCHUSDT.fut',
    'id': 1687,
    'name': 'BCHUSDT.fut'},
   {'groupId': 16,
    'siteName': 'BNB',
    'turnOn': True,
    'description': 'BNBUSDT',
    'id': 1688,
    'name': 'BNBUSDT.fut'},
   {'groupId': 16,
    'siteName': 'Ethereum',
    'turnOn': True,
    'description': 'ETHUSDT',
    'id': 1690,
    'name': 'ETHUSDT.fut'},
   {'groupId': 16,
    'siteName': 'Litecoin',
    'turnOn': True,
    'description': 'LTCUSDT',
    'id': 1692,
    'name': 'LTCUSDT.fut'},
   {'groupId': 16,
    'siteName': 'TRON',
    'turnOn': True,
    'description': 'TRXUSDT',
    'id': 1693,
    'name': 'TRXUSDT.fut'},
   {'groupId': 16,
    'siteName': 'Cardano',
    'turnOn': True,
    'description': 'ADAUSDT',
    

In [42]:
result = {}
for row in bondash_data:
    for insrument in row['instruments']:
        if insrument['name']:
            if insrument['name'].replace("#", '').strip() in ticket:
                result[insrument['name'].replace("#", '').strip()] =  insrument['id']
        
result

{'EURUSD': 1,
 'AUDUSD': 6,
 'EURCHF': 13,
 'EURGBP': 15,
 'GBPJPY': 32,
 'GBPUSD': 37,
 'NZDUSD': 42,
 'USDCAD': 44,
 'USDCHF': 45,
 'USDJPY': 50}

In [25]:
set(result.keys()) ^ set(ticket) # таких инструментов нет()

set()

In [45]:
# Преобразуем результат в DataFrame
result_df = pd.DataFrame(list(result.items()), columns=['Ticket', 'value'])

# Теперь выполняем merge
unity_instruments = unity_instruments.merge(result_df, on='Ticket', how='left')
unity_instruments.rename(columns={"ID": "id_unity", "value": "id_bondash"}, inplace=True)
unity_instruments.head()

,id_unity,Ticket,id_bondash
0,47,EURUSD,1
1,37,GBPUSD,37
2,41,USDJPY,50
3,30,AUDUSD,6
4,64,USDCAD,44


In [46]:
unity_instruments.tail()

,id_unity,Ticket,id_bondash
5,12,NZDUSD,42
6,36,USDCHF,45
7,76,EURGBP,15
8,26,EURCHF,13
9,35,GBPJPY,32


In [24]:

unity_instruments[["id_unity", "id_bondash", "Ticket"]].dropna().to_csv('id_instruments_prod.csv', index=False)


In [48]:
unity_instruments[["id_unity", "id_bondash", "Ticket"]].dropna().to_csv('id_instruments_prod.csv', index=False, mode='a', header=False)


In [25]:
df_ = unity_instruments[["id_unity", "id_bondash", "Ticket"]].dropna()
id_unity = df_["id_unity"].dropna().astype(int).tolist()
id_bondash = df_["id_bondash"].dropna().astype(int).tolist()
ticket = df_["Ticket"].tolist()


In [26]:
d = dict(zip(id_bondash, id_unity))
id_trans_to_tiket = dict(zip(id_bondash, ticket))
id_trans_to_tiket

{191: 'AAPL',
 238: 'NVDA',
 233: 'MSFT',
 187: 'AMZN',
 217: 'GOOG',
 250: 'TSLA',
 375: 'AVGO',
 380: 'LLY',
 256: 'WMT',
 226: 'JPM',
 255: 'V',
 229: 'MA',
 239: 'ORCL',
 317: 'UNH',
 210: 'XOM',
 204: 'COST',
 236: 'NFLX',
 220: 'HD',
 240: 'PG',
 1666: 'JNJ',
 193: 'BAC',
 300: 'ABBV',
 199: 'CAT',
 200: 'CVX',
 257: 'WFC',
 1677: 'PLTR',
 201: 'CSCO',
 184: 'BABA',
 433: 'IBM',
 228: 'MS',
 244: 'PM',
 173: 'ABT',
 231: 'MRK',
 230: 'MCD',
 361: 'ISRG'}

In [27]:


# URL для запроса
url = "https://clientapi.bondah.com/articles"

# Заголовки запроса
headers = {
    "LanguageId": "7",  # Укажите нужный язык, например 'en' для английского
    # "LanguageId": "1", 
}

# # Параметры запроса
params = {
    "FromId": None,           # ID статьи, после которой получить статьи (если не нужно, оставить None)
    "ArticlesCount": 5,      # Количество статей (по умолчанию 10)
    "InstrumentId": list(map(int, d.keys())),
    # d.keys(),
    "IsSignals": True

}

# # Удаляем None из параметров, чтобы не отправлять их
# params = {k: v for k, v in params.items() if v is not None}

# Выполнение GET-запроса
try:
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # Проверка на успешность запроса
    articles = response.json()  # Декодирование ответа в JSON
    print("Статьи получены:", articles)
except requests.exceptions.RequestException as e:
    print("Ошибка выполнения запроса:", e)



Статьи получены: [{'id': 34910828, 'instrumentId': 191, 'analyseTypeId': 2, 'analyseType': 'Double Tops', 'snapshotId': 7199, 'header': 'Apple M30: Видны сигналы затухания восходящего движения', 'body': 'На графике Apple видны признаки слабости восходящего движения, поскольку цена не может преодолеть уровень сопротивления.<br/>Формируется классический разворотный паттерн Двойная вершина на графике. Пробой линии шеи, на уровне 242.92, служит основным сигналом для подтверждения. Дальше мы можем рассматривать два возможных сценария развития событий. Первый сценарий предполагает закрепление цены ниже линии шеи и дальнейшее снижение цены. В рамках второго сценария цена может вернуться выше линии шеи, что может привести к отмене паттерна и продолжению движения вверх.<br/>В этой ситуации приоритет отдаётся коротким позициям.<br/>📌 Вход: 242.80<br/>✔️ Первая цель: 240.23<br/>✔️ Вторая цель: 238.44<br/>❌ Стоп-ордер: 245.16<br/>', 'createTime': '2025-02-18T19:01:15', 'timeAgo': '22 дня назад', '

In [42]:
import requests

# URL для запроса
url = "https://clientapi.bondah.com/articles"

# Параметры запроса
params = {
    "FromId": None,
    "ArticlesCount": 5,
    "InstrumentId": d.keys(),
    "IsSignals": True
}

# Список языков для запроса
languages = [7, 1]  # RU и EN или другие идентификаторы языков

# Хранилище для результатов
all_articles = []

# Выполнение запросов для каждого языка
for lang in languages:
    headers = {"LanguageId": str(lang)}
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Проверка на успешность запроса
        articles = response.json()  # Декодирование ответа в JSON
        all_articles.append(articles)  # Добавляем результаты в общий список
        print(f"Статьи для языка {lang} получены:", articles)
    except requests.exceptions.RequestException as e:
        print(f"Ошибка выполнения запроса для языка {lang}:", e)

# Итоговые объединённые статьи
print("Все статьи:", all_articles)


Статьи для языка 7 получены: [{'id': 34910828, 'instrumentId': 191, 'analyseTypeId': 2, 'analyseType': 'Double Tops', 'snapshotId': 7199, 'header': 'Apple M30: Видны сигналы затухания восходящего движения', 'body': 'На графике Apple видны признаки слабости восходящего движения, поскольку цена не может преодолеть уровень сопротивления.<br/>Формируется классический разворотный паттерн Двойная вершина на графике. Пробой линии шеи, на уровне 242.92, служит основным сигналом для подтверждения. Дальше мы можем рассматривать два возможных сценария развития событий. Первый сценарий предполагает закрепление цены ниже линии шеи и дальнейшее снижение цены. В рамках второго сценария цена может вернуться выше линии шеи, что может привести к отмене паттерна и продолжению движения вверх.<br/>В этой ситуации приоритет отдаётся коротким позициям.<br/>📌 Вход: 242.80<br/>✔️ Первая цель: 240.23<br/>✔️ Вторая цель: 238.44<br/>❌ Стоп-ордер: 245.16<br/>', 'createTime': '2025-02-18T19:01:15', 'timeAgo': '22 д

In [43]:
d

{191: 8115,
 238: 4711,
 233: 1370,
 187: 8191,
 217: 795,
 250: 1291,
 375: 3109,
 380: 693,
 256: 9115,
 226: 7582,
 255: 7320,
 229: 7137,
 239: 4957,
 317: 4476,
 210: 3278,
 204: 7654,
 236: 6359,
 220: 9504,
 240: 9213,
 1666: 6670,
 193: 8940,
 300: 3924,
 199: 1155,
 200: 5573,
 257: 5033,
 1677: 6396,
 201: 1623,
 184: 5213,
 433: 8254,
 228: 1139,
 244: 8634,
 173: 8050,
 231: 6381,
 230: 2053,
 361: 3768}

In [44]:
ru_df = pd.DataFrame(all_articles[0])
en_df = pd.DataFrame(all_articles[1])
ru_df['header_en'] = en_df['header'].copy()
ru_df['body_en'] = en_df['body'].copy()
ru_df['instrumentId'] = ru_df['instrumentId']
ru_df

,id,instrumentId,analyseTypeId,analyseType,snapshotId,header,body,createTime,timeAgo,foreCast,mtPicture,header_en,body_en
0,34910828,191,2,Double Tops,7199,Apple M30: Видны сигналы затухания восходящего...,На графике Apple видны признаки слабости восхо...,2025-02-18T19:01:15,22 дня назад,Down,UklGRlYgAABXRUJQVlA4TEkgAAAv/8THAJ/iNrZtVVkfid...,Apple M30: Signals of a decay of the upward tr...,The Apple chart shows signs of weakness in the...
1,26691277,191,11,Wolfe's Waves,12162,Apple H1: Происходит формирование Волн Вульфа,Расстановка сил по Apple постепенно улучшается...,2025-01-10T15:01:24,2 месяца назад,Up,R0lGODlhAAUgA/cAAAAAAAAAMwAAZgAAmQAAzAAA/wArAA...,Apple H1: Wolfe Waves are forming,The balance of power in Apple is gradually imp...
2,26691069,191,11,Wolfe's Waves,12158,Apple H1: Формирование пятой волны Вульфа указ...,Apple продолжает движение вниз. При этом на гр...,2025-01-10T15:00:36,2 месяца назад,Up,UklGRpQtAABXRUJQVlA4TIctAAAv/8THAEfjNrZtVdnf3T...,Apple H1: The formation of the fifth Wolfe wav...,Apple continues to move down. At the same time...
3,26166025,191,9,Triangle,8902,Apple M15: Сигнал на снижение определён на гра...,"Расстановка сил по Apple постепенно меняется, ...",2025-01-07T16:46:26,2 месяца назад,Down,UklGRmAhAABXRUJQVlA4TFQhAAAv/8THAP+gJLat6H5NBO...,Apple M15: The downward signal is visualized o...,The balance of power in Apple is gradually cha...
4,26165973,191,9,Triangle,8901,Apple M15: Сформирован сигнал на снижение,Сформированный на графике Apple паттерн свид...,2025-01-07T16:45:41,2 месяца назад,Down,R0lGODlhAAUgA/cAAAAAAAAAMwAAZgAAmQAAzAAA/wArAA...,Apple M15: A downward signal has been generated,The Apple pattern formed on the chart indicate...


In [55]:
def get_data() -> list[list]:
    bondah_url = "https://clientapi.bondah.com/articles"
    headers = {"LanguageId": "7"}
    params = {
        "FromId": None,
        "ArticlesCount": 10,
        "InstrumentId": reaL_value.keys(),
        "IsSignals": True
    }
    languages = [7, 1]  # RU и EN или другие идентификаторы языков

    # Хранилище для результатов
    all_articles = []

    # Выполнение запросов для каждого языка
    for lang in languages:
        headers = {"LanguageId": str(lang)}
        try:
            response = requests.get(bondah_url, headers=headers, params=params)
            response.raise_for_status()  # Проверка на успешность запроса
            articles = response.json()  # Декодирование ответа в JSON
            all_articles.append(articles)  # Добавляем результаты в общий список
        except requests.exceptions.RequestException as e:
            # log_message(f"Ошибка выполнения запроса для языка {lang}:", e)

            return []
    return all_articles
all_articles = get_data()

In [57]:
all_articles

[[{'id': 37623980,
   'instrumentId': 1,
   'analyseTypeId': 1,
   'analyseType': 'Divergences',
   'snapshotId': 115317,
   'header': 'EURUSD M15: Дивергенция на снижение от RSI',
   'body': 'Осциллятор RSI выявляет несоответствие. Цена не смогла достичь нового пика, в то время как индикатор RSI поднялся выше предыдущего максимума, указывая на потенциальное изменение направления движения цены.<br/>Гистограмма индикатора MACD удерживается выше сигнальной линии, указывая на локальное развитие восходящего движения. Тем не менее, показания индикатора не противоречат сигналу отмеченному на графике.<br/>На индикаторе Stochastic голубая главная линия находится выше сигнальной красной. Разворотный паттерн пока не сформирован.<br/>В этой ситуации для торговли EURUSD стоит рассматривать короткие позиции.<br/>📌 Вход: 1.09089<br/>✔️ Первая цель: 1.08873<br/>✔️ Вторая цель: 1.08675<br/>❌ Стоп-ордер: 1.09228<br/>',
   'createTime': '2025-03-12T08:37:18',
   'timeAgo': 'вчера',
   'foreCast': 'Down'

In [64]:
text = all_articles[0][0]['body']
print(text)

Осциллятор RSI выявляет несоответствие. Цена не смогла достичь нового пика, в то время как индикатор RSI поднялся выше предыдущего максимума, указывая на потенциальное изменение направления движения цены.<br/>Гистограмма индикатора MACD удерживается выше сигнальной линии, указывая на локальное развитие восходящего движения. Тем не менее, показания индикатора не противоречат сигналу отмеченному на графике.<br/>На индикаторе Stochastic голубая главная линия находится выше сигнальной красной. Разворотный паттерн пока не сформирован.<br/>В этой ситуации для торговли EURUSD стоит рассматривать короткие позиции.<br/>📌 Вход: 1.09089<br/>✔️ Первая цель: 1.08873<br/>✔️ Вторая цель: 1.08675<br/>❌ Стоп-ордер: 1.09228<br/>


In [65]:
import requests
import pandas as pd
import re
from datetime import datetime, timedelta
import json
import time
import random 
# Словарь соответствий инструментов
df_id = pd.read_csv('id_instruments_prod.csv')
id_unity = df_id['id_unity'].astype(int).tolist()
id_bondash = df_id['id_bondash'].astype(int).tolist()
ticket = df_id["Ticket"].tolist()
reaL_value = dict(zip(id_bondash, id_unity))
id_trans_to_ticket = dict(zip(id_bondash, ticket))
find_instr = dict(zip(id_unity, ticket))
# reaL_value = {1: 47, 1686: 61343, 191: 8115, 250: 1291, 238: 4711}


TOKEN = '6766212755:AAGNwQDGrFoICY4s3WwHLBY0tH55AY6U01c'
URL = 'https://api.telegram.org/bot'
chat_id = -4017930976


def send_message(text:str):
    requests.get('https://api.telegram.org/bot{}/sendMessage'.format(TOKEN),
                       params={'chat_id': chat_id, 'text': text})
    


# URL и токен для запросов
url = 'https://rest.unity.finance/api/v1/createTradeIdea'
token = '666beb18-942a-46e9-87d7-e327905d6219' # prod U

headers = {
    "accept": "application/json",
    "auth-token": token,
    'Content-Type': 'application/json; charset=utf-8'
}

# Логирование
log_file = "logfile.log"
def log_message(message):
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

# def translane(text) ->str:
#     base_url = "https://ftapi.pythonanywhere.com/translate"

#     params = {
#         "sl": "ru",  # Исходный язык (русский)
#         "dl": "en",  # Целевой язык (английский)
#         "text": text # Текст для перевода
#     }

#     # Выполнение GET-запроса
#     response = requests.get(base_url, params=params)

#     # Проверка статуса ответа
#     if response.status_code == 200:
#         translation_data = response.json()
#         return  translation_data["destination-text"]
#     else:
#         log_message('Ошибка в переводе. Не удалось перевести текст')
#         return " "


# Функция для преобразования времени
def make_correct_time(time: str, delta: int = 0) -> str:
    date_obj = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    date_obj += timedelta(hours=delta)
    return date_obj.strftime('%Y-%m-%dT%H:%M:%SZ')



def escape_whitespace(text: str) -> str:
    """
    Экранирует управляющие символы в тексте с помощью json.dumps.
    json.dumps возвращает строку в виде JSON-литерала (с обрамляющими кавычками),
    поэтому убираем первую и последнюю кавычки.
    """
    return json.dumps(text)[1:-1]

def extract_trading_levels_from_escaped(escaped_text: str):
    """
    Извлекает торговые уровни из экранированного текста.
    Допускается наличие пробельных символов и экранированных последовательностей (например, "\\n")
    между ключевыми словами и значениями.
    """
    # Разрешаем пробелы и последовательности "\n"
    sep = r'(?:\s|\\n)*'
    patterns = {
        'entry': re.compile(r'\b(?:Вход|Entry):' + sep + r'(\d+\.\d+)'),
        'tp1': re.compile(r'\b(?:Первая цель|TP1):' + sep + r'(\d+\.\d+)'),
        'tp2': re.compile(r'\b(?:Вторая цель|TP2):' + sep + r'(\d+\.\d+)'),
        'sl': re.compile(r'\b(?:Стоп-ордер|SL):' + sep + r'(\d+\.\d+)')
    }
    
    levels = {}
    for key, pattern in patterns.items():
        match = pattern.search(escaped_text)
        levels[key] = float(match.group(1)) if match else None
    return levels

def escape_and_extract_levels(text: str):
    """
    Сначала экранирует управляющие символы в тексте,
    а затем извлекает из него торговые уровни.
    """
    escaped_text = escape_whitespace(text)
    return extract_trading_levels_from_escaped(escaped_text)

def escape_and_extract_levels(text: str):
    """
    Сначала экранирует текст, а затем извлекает уровни из экранированного текста.
    """
    escaped_text = escape_whitespace(text)
    return extract_trading_levels_from_escaped(escaped_text)


# Получение данных
def get_data() -> list[list]:
    bondah_url = "https://clientapi.bondah.com/articles"
    headers = {"LanguageId": "7"}
    params = {
        "FromId": None,
        "ArticlesCount": 10,
        "InstrumentId": reaL_value.keys(),
        "IsSignals": True
    }
    languages = [7, 1]  # RU и EN или другие идентификаторы языков

    # Хранилище для результатов
    all_articles = []

    # Выполнение запросов для каждого языка
    for lang in languages:
        headers = {"LanguageId": str(lang)}
        try:
            response = requests.get(bondah_url, headers=headers, params=params)
            response.raise_for_status()  # Проверка на успешность запроса
            articles = response.json()  # Декодирование ответа в JSON
            all_articles.append(articles)  # Добавляем результаты в общий список
        except requests.exceptions.RequestException as e:
            log_message(f"Ошибка выполнения запроса для языка {lang}:", e)
            return []
    return all_articles

def escape_all(text: str) -> str:
    """
    Сначала заменяет '<br/>' на '\\n', затем экранирует все управляющие символы
    с помощью json.dumps (убираются лишние обрамляющие кавычки).
    """
    # Заменяем HTML-разрывы строк на символы переноса строки в виде последовательности "\n"
    text = text.replace('<br/>', '\\n')
    # Экранируем оставшиеся управляющие символы (например, реальные переводы строки)
    return json.dumps(text)[1:-1]

In [80]:
# Обработка данных
def process_data(all_articles: list):
    if not all_articles:
        
        return
    
    ru_df = pd.DataFrame(all_articles[0])
    en_df = pd.DataFrame(all_articles[1])
    ru_df['header_en'] = en_df['header'].copy()
    ru_df['body_en'] = en_df['body'].copy()
    df = ru_df.copy()

    df['createTime'] = pd.to_datetime(df['createTime'])
    df.drop(['analyseTypeId', 'snapshotId', 'timeAgo', 'mtPicture'], axis=1, inplace=True, errors='ignore')

    df_id = pd.read_csv('check_id.csv')
    new_df = df[df.id > df_id.id[0]].copy()
    if new_df.empty:
        log_message("Нет новых записей для обработки.")
        return

    df_id.id = df.id.max()
    df_id.to_csv('check_id.csv', index=False)

    new_df[['entry', 'tp1', 'tp2', 'sl']] = new_df['body'].apply(lambda x: pd.Series(extract_trading_levels_from_escaped(x)))
    new_df['instrumentId'] = new_df['instrumentId'].replace(reaL_value)
    new_df['createTime'] = new_df['createTime'].astype(str)
    new_df['foreCast'] = new_df['foreCast'].replace({'Down': 'SELL', 'Up': 'BUY'})
    new_df['expirationTime'] = new_df['createTime'].apply(lambda x: make_correct_time(time=x, delta=24))
    new_df['createTime'] = new_df['createTime'].apply(lambda x: make_correct_time(x))

    for index, row in new_df.iterrows():
        print(row)
        en_header = row['header_en']
        en_body = row['body_en']
        target = {
            "instrumentIds": [row['instrumentId']],
            "openOrderType": "LIMIT",
            "side": "BUY",
            "price": row['entry'],
            "stopLoss": row['sl'],
            "takeProfit": row['tp1'],
            "confidence": f'{random.randint(55, 90) / 100}',
            "expirationTime": "2025-03-15T08:37:18Z",
            "localeDescriptions": [
               {
                "locale": "RU",
                "target": (row['header']),
                "background": (row['body']).replace('<br/>', '\n')
            },
            {
                "locale": "EN",
                "target": (en_header),
                "background": (en_body).replace('<br/>', '\n')
            }
            ],
            "status": "ACTIVE",
            "publishTime": row['createTime'],
            "source": "Bondah Analytics",
            "provider": "UNITY",
            "state": "LIVE_TRADE",
            "externalId": str(row['id']),
            "subscribers": 0,
            "priority": 0,
            "rate": 0,
            "confidence": f'{random.randint(55, 90) / 100}',
        }

        payload = json.dumps(target, indent=4, ensure_ascii=False)
        return(payload)

In [81]:
dat = process_data(all_articles)
dat

id                                                         37623980
instrumentId                                                      1
analyseType                                             Divergences
header                   EURUSD M15: Дивергенция на снижение от RSI
body              Осциллятор RSI выявляет несоответствие. Цена н...
createTime                                     2025-03-12T08:37:18Z
foreCast                                                       SELL
header_en              EURUSD M15: The downward divergence from RSI
body_en           The RSI oscillator detects a discrepancy. The ...
entry                                                       1.09089
tp1                                                         1.08873
tp2                                                         1.08675
sl                                                          1.09228
expirationTime                                 2025-03-13T08:37:18Z
Name: 0, dtype: object


'{\n    "instrumentIds": [\n        1\n    ],\n    "openOrderType": "LIMIT",\n    "side": "BUY",\n    "price": 1.09089,\n    "stopLoss": 1.09228,\n    "takeProfit": 1.08873,\n    "confidence": "0.87",\n    "expirationTime": "2025-03-15T08:37:18Z",\n    "localeDescriptions": [\n        {\n            "locale": "RU",\n            "target": "EURUSD M15: Дивергенция на снижение от RSI",\n            "background": "Осциллятор RSI выявляет несоответствие. Цена не смогла достичь нового пика, в то время как индикатор RSI поднялся выше предыдущего максимума, указывая на потенциальное изменение направления движения цены.\\nГистограмма индикатора MACD удерживается выше сигнальной линии, указывая на локальное развитие восходящего движения. Тем не менее, показания индикатора не противоречат сигналу отмеченному на графике.\\nНа индикаторе Stochastic голубая главная линия находится выше сигнальной красной. Разворотный паттерн пока не сформирован.\\nВ этой ситуации для торговли EURUSD стоит рассматрив

In [86]:
print( "The RSI oscillator detects a discrepancy. The price failed to reach a new peak, while the RSI indicator rose above the previous intraday high, indicating a potential change in price movement dynamics.\nThe histogram of the MACD indicator remains above the signal line, indicating the local development of an upward movement. However, the indicator indications do not contradict the signal, marked on the chart.\nThere is the blue main line above the red signal line on the Stochastic indicator. The reversal pattern has not formed yet.\nIn this situation, it is worth considering short positions to trade EURUSD.\n📌 Entry: 1.09089\n✔️ First target: 1.08873\n✔️ Second target: 1.08675\n❌ Stop order: 1.09228\n")

The RSI oscillator detects a discrepancy. The price failed to reach a new peak, while the RSI indicator rose above the previous intraday high, indicating a potential change in price movement dynamics.
The histogram of the MACD indicator remains above the signal line, indicating the local development of an upward movement. However, the indicator indications do not contradict the signal, marked on the chart.
There is the blue main line above the red signal line on the Stochastic indicator. The reversal pattern has not formed yet.
In this situation, it is worth considering short positions to trade EURUSD.
📌 Entry: 1.09089
✔️ First target: 1.08873
✔️ Second target: 1.08675
❌ Stop order: 1.09228



In [83]:

def test(data):

    url = 'https://rest.portal.stage.unityfinance.net/api/v1/createTradeIdea'
    token = 'e8fb0caa-62b0-4fa5-8f89-4c71ff0a5afc' # prod U

    headers = {
        "accept": "application/json",
        "auth-token": token,
        'Content-Type': 'application/json; charset=utf-8'
    }

    response = requests.post(url, headers=headers, data=dat.encode('utf-8'))
    return response

response

<Response [200]>